In [1]:
# Because we don't want to build a notebook image with kafka-python, we have to install it each time.

!pip install kafka-python --index-url http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

    100% |████████████████████████████████| 266kB 7.4MB/s 
You are using pip version 8.1.0, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Cluster Metadata

In [24]:
from kafka.cluster import ClusterMetadata

servers = ['kafka1:19092']

metadata = ClusterMetadata(bootstrap_servers=servers)

print(metadata)

ClusterMetadata(brokers: 0, topics: 0, groups: 0)


# Producer

In [15]:
import os
import logging
import time
import uuid
from datetime import datetime

import kafka
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic

BOOTSTRAP_SERVERS = os.getenv("BOOTSTRAP_SERVERS", "172.25.0.21:9092")

# change logging config
logging.basicConfig(level=logging.INFO,
                    format='[%(asctime)s.%(msecs)03d][%(filename)s:%(lineno)d][%(levelname)s]%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

logger = logging.getLogger(__name__)

try:                                                        
    producer_delay = float(os.getenv('PRODUCER_DELAY', 1.0))
except:                                                     
    producer_delay = 1.0                                    
logger.info('Producer delay: %s', producer_delay)

def create_test_topic():
    client_id = 'producer-{0}'.format(str(uuid.uuid4()))
    admin_client = KafkaAdminClient(bootstrap_servers=BOOTSTRAP_SERVERS, client_id=client_id)

    topic_list = []
    topic_list.append(NewTopic(name="test", num_partitions=1, replication_factor=1))
    admin_client.create_topics(new_topics=topic_list, validate_only=False)
    
def send_message():
    producer = KafkaProducer(bootstrap_servers=BOOTSTRAP_SERVERS)
    counter = 0
    while True:
        body = '{0}:{1}'.format(datetime.now().strftime(
                '%Y-%m-%d %H:%M:%S.%f'), str(uuid.uuid4()))
        producer.send('test', str.encode(body))
        
        counter += 1
        if counter % 500 == 0:
            logger.info('Sent %s messages.', str(counter))
#         time.sleep(0.1)

try:
    create_test_topic()
except Exception:
    pass

send_message()

KeyboardInterrupt: 